In [115]:
import numpy as np

from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.core.Datasets import Datasets
from ptrail.features.kinematic_features import KinematicFeatures as kinematic
from ptrail.features.temporal_features import TemporalFeatures as temporal
from ptrail.preprocessing.filters import Filters as filters
import ptrail.utilities.constants as const

import pandas as pd
import time
import matplotlib.pyplot as plt
np.seterr(invalid='ignore')
start = time.time()

In [116]:
%%time
"""
    First of all, lets import all the datasets one by one
    and check out a few of their points.
"""

df = pd.read_csv('../../../Data/OpenSky/states_2022-06-27-01.csv', delimiter=',')
#df = pd.read_parquet('../../../Data/city=Singapore/part-00003-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='auto')

df


CPU times: user 4.21 s, sys: 446 ms, total: 4.66 s
Wall time: 4.8 s


,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,1656291600,40717b,48.703217,12.926331,216.466030,314.133407,0.00000,EXS94FR,False,False,False,3551.0,11582.40,12047.22,1.656292e+09,1.656292e+09
1,1656291600,a3d03a,47.424271,-122.165909,51.511234,8.615648,0.32512,N345DP,False,False,False,1200.0,807.72,822.96,1.656292e+09,1.656292e+09
2,1656291600,a1210a,37.354294,-122.029938,56.994266,329.036243,-2.60096,N362Q,False,False,False,332.0,563.88,556.26,1.656292e+09,1.656292e+09
3,1656291600,a1ea70,37.548706,-121.959696,197.557883,355.818713,14.63040,SWA2082,False,False,False,3374.0,5090.16,5372.10,1.656292e+09,1.656292e+09
4,1656291600,3c4581,48.738610,14.705489,263.913783,125.239062,0.00000,BOX1576,False,False,False,6414.0,10058.40,10507.98,1.656292e+09,1.656292e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084070,1656295190,a3be8f,36.517717,-78.056108,190.186405,24.277211,0.00000,DAL2598,False,False,False,5673.0,7620.00,8061.96,1.656295e+09,1.656295e+09
2084071,1656295190,4b5cdf,NaN,NaN,NaN,NaN,NaN,TEST1234,False,False,False,NaN,NaN,NaN,NaN,1.656295e+09
2084072,1656295190,4bb161,28.399146,76.894549,152.179805,291.621574,4.55168,THY717,False,False,False,NaN,3139.44,3223.26,1.656295e+09,1.656295e+09
2084073,1656295190,a4b241,32.302784,-110.284629,249.882667,94.842008,0.00000,TWY22,False,False,False,3716.0,13723.62,14439.90,1.656295e+09,1.656295e+09


In [117]:
dict_df = PTRAILDataFrame(data_set=df,
                        latitude='lat',
                        longitude='lon',
                        datetime='time',
                        traj_id='icao24')
print(f"The dimensions of the dataframe:{dict_df.shape}")
print(f"Type of the dataframe: {type(dict_df)}")

The dimensions of the dataframe:(2084075, 14)
Type of the dataframe: <class 'ptrail.core.TrajectoryDF.PTRAILDataFrame'>


In [118]:
atlantic = kinematic.create_speed_column(dict_df)


/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3

In [119]:
max_speed_filt_df = filters.filter_by_max_speed(atlantic, 1000)
print(f"Length of atlantic: {len(atlantic)}")
print(f"Length of speed_filt_df: {len(max_speed_filt_df)}")

Length of atlantic: 2084075
Length of speed_filt_df: 385529


In [120]:
# Now, lets filter the dataframe based on minimum speed.

min_speed_filt = filters.filter_by_min_speed(max_speed_filt_df, 0)
print(f"Length of speed_filt_df: {len(max_speed_filt_df)}")
print(f"Length of min_speed_filt: {len(min_speed_filt)}")

Length of speed_filt_df: 385529
Length of min_speed_filt: 385529


In [121]:
odf_two = filters.filter_outliers_by_consecutive_speed(min_speed_filt)
print(f"length of geolife: {len(atlantic)}")
print(f"length of outlier_df: {len(odf_two)}")
print(f"Number of outliers: {len(atlantic) - len(odf_two)}")

length of geolife: 2084075
length of outlier_df: 0
Number of outliers: 2084075


In [122]:
odf_two.to_csv('../Output/OpenSky-states_2022-06-27-01LAST.csv')